### Purpose of this Notebook:

**Input:** Predicted .txt files.

**Output:** A .csv file containing evaluation metrics such as mAP@0.50:0.95, mAP@0.50, precision, recall, F1 score, total number of annotations (n_all), number of false negatives (n_fn), number of false positives (n_fp), configuration details, model name, dataset size (n), and iteration number.

**Description:** This notebook is designed for processing predicted .txt files generated by object detection models. The naming convention of these files (e.g., yolo_nas_l_32_1_labelsPred.txt) includes information about the model used (e.g., yolo_nas_l, yolo_nas_m, yolo_nas_s), the size of the data sample (e.g., 16, 32, 64, 128, 256, 250, 500), and that the file contains predicted labels. The notebook loads these files and calculates several key evaluation metrics. The output is a .csv file that organizes these metrics along with additional information such as model configuration, dataset size, and iteration number, facilitating an easy comparison across different model runs and configurations.

In [1]:
import os
os.chdir("..")
ROOT = os.getcwd()
os.environ['TORCH_HOME'] = '/home/mautushid/.torch' #only if running notebook on ood
from evaluate import from_sv
from API import *
import pandas as pd

Matplotlib created a temporary cache directory at /localscratch/2212733/matplotlib-fy9f9jcu because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
/localscratch/2212733/ipykernel_3251461/394336794.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


**Description:** 

##### make_table_row:

**Input:** Evaluation metrics (EvalOutput dictionary), dataset name, model name, data size, iteration number.

**Output:** A single-row pandas DataFrame combining all inputs, ready for CSV export.

#### initialize_csv:

**Input:** File path, column names for the CSV.

**Output:** An empty CSV file created at the specified path with the given column names.

##### append_row_to_csv:

**Input:** File path of the CSV, a single-row pandas DataFrame to append.

**Output:** The input DataFrame is appended to the CSV file at the specified path, expanding the file's data without repeating headers.

In [2]:
def make_table_row(EvalOutput, dataset_name, model_name, data_size, iter_num):
    # Combine EvalOutput with the additional information
    combined_dict = {
        **EvalOutput,  # Unpack the EvalOutput dictionary
        'config': dataset_name,  # Add dataset_name as 'config'
        'model': model_name,  # Add model_name as 'model'
        'n': data_size,  # Add data_size as 'n'
        'iteration': iter_num}

    # Create a DataFrame from the combined dictionary
    # The index=[0] makes the dictionary values to be considered as a single row
    df = pd.DataFrame([combined_dict])

    return df

def initialize_csv(file_path, column_names):
    # Create an empty DataFrame with the specified column names
    df_empty = pd.DataFrame(columns=column_names)
    # Write the empty DataFrame to a CSV file, specifying the header
    df_empty.to_csv(file_path, index=False)
def append_row_to_csv(file_path, row_df):
    # Append the row to the CSV file without the header and without the index
    row_df.to_csv(file_path, mode='a', header=False, index=False)


##### Creating CSV 

In [3]:
### defining variables
yolo_bases = ['yolo_nas_l', 'yolo_nas_m', 'yolo_nas_s']
configs = ["1a_angle_t2s", "1b_angle_s2t", "2_light", "3_breed", "4_all"]
nsa = [32, 64, 128, 256, 500]  # Including the complete set
nsb = [32, 64, 128, 250] 
iterations = range(1, 10)  # Adjust the range as needed


### initializing empty output csv file
csv_file_path = ROOT + '/table4.csv'
column_names = ['map5095','map50','precision','recall','f1',\
                'n_all','n_fn','n_fp','config','model','n','iteration']
initialize_csv(csv_file_path,column_names)


def main(csv_file_path, yolo_base, config, n, iteration):
    
    config_short = config.split("_")[-1]
    gt_label_path = ROOT + "/data/"+config+"/test/labels"
    gt_img_path = ROOT + "/data/"+config+"/test"
    
    pred_label_path = ROOT+"/data/"+config+"/test/"+"exp"+"_"+yolo_base+"_"+str(n)+"_"+str(iteration)+"_labelsPred"
    
    api_path = ROOT+"/data/"+config+"/tv/"+"exp_"+yolo_base+"_"+str(n)+"_"+str(iteration)+"_"+\
                config_short+"_"+yolo_base+"_"+str(n)+"_"+str(iteration)

    api = YOLO_API(api_path)
    #lbs = api.get_gt_detections(gt_label_path, gt_img_path)
    #pre = api.get_pred_detections(pred_label_path)
    
    lbs,pre = api.get_gt_pred_detections(gt_img_path,gt_label_path,pred_label_path)

    EvalOutput = from_sv(pre, lbs) #dict with keys 'map5095','map50','precision','recall','f1','n_all','n_fn''n_fp'
    data_point = make_table_row(EvalOutput, config, yolo_base, n, iteration) ## will create a dataframe
    append_row_to_csv(csv_file_path, data_point)

In [4]:
# Iterate through each combination of parameters
for yolo_base in yolo_bases:
    for config in configs:
        if config == '3_breed':
            ns = nsb
        else:
            ns = nsa
        for n in ns:
            for iteration in range(1,25):
                print(f"Processing: yolo_base={yolo_base}, config={config}, n={n}, iteration={iteration}")
                # Call the main function with the current combination of parameters
                try:
                    main(csv_file_path, yolo_base, config, n, iteration)
                except Exception as e:
                    print(f"Error processing {yolo_base}, {config}, n={n}, iteration={iteration}: {e}")
                    continue

Processing: yolo_base=yolo_nas_l, config=1a_angle_t2s, n=32, iteration=1
Processing: yolo_base=yolo_nas_l, config=1a_angle_t2s, n=32, iteration=2
Processing: yolo_base=yolo_nas_l, config=1a_angle_t2s, n=32, iteration=3
Processing: yolo_base=yolo_nas_l, config=1a_angle_t2s, n=32, iteration=4
Processing: yolo_base=yolo_nas_l, config=1a_angle_t2s, n=32, iteration=5
Processing: yolo_base=yolo_nas_l, config=1a_angle_t2s, n=32, iteration=6
Processing: yolo_base=yolo_nas_l, config=1a_angle_t2s, n=32, iteration=7
Processing: yolo_base=yolo_nas_l, config=1a_angle_t2s, n=32, iteration=8
Processing: yolo_base=yolo_nas_l, config=1a_angle_t2s, n=32, iteration=9
Processing: yolo_base=yolo_nas_l, config=1a_angle_t2s, n=32, iteration=10
Processing: yolo_base=yolo_nas_l, config=1a_angle_t2s, n=32, iteration=11
Processing: yolo_base=yolo_nas_l, config=1a_angle_t2s, n=32, iteration=12
Processing: yolo_base=yolo_nas_l, config=1a_angle_t2s, n=32, iteration=13
Processing: yolo_base=yolo_nas_l, config=1a_ang

Processing: yolo_base=yolo_nas_l, config=1a_angle_t2s, n=256, iteration=2
Processing: yolo_base=yolo_nas_l, config=1a_angle_t2s, n=256, iteration=3
Processing: yolo_base=yolo_nas_l, config=1a_angle_t2s, n=256, iteration=4
Processing: yolo_base=yolo_nas_l, config=1a_angle_t2s, n=256, iteration=5
Processing: yolo_base=yolo_nas_l, config=1a_angle_t2s, n=256, iteration=6
Processing: yolo_base=yolo_nas_l, config=1a_angle_t2s, n=256, iteration=7
Processing: yolo_base=yolo_nas_l, config=1a_angle_t2s, n=256, iteration=8
Processing: yolo_base=yolo_nas_l, config=1a_angle_t2s, n=256, iteration=9
Processing: yolo_base=yolo_nas_l, config=1a_angle_t2s, n=256, iteration=10
Processing: yolo_base=yolo_nas_l, config=1a_angle_t2s, n=256, iteration=11
Processing: yolo_base=yolo_nas_l, config=1a_angle_t2s, n=256, iteration=12
Processing: yolo_base=yolo_nas_l, config=1a_angle_t2s, n=256, iteration=13
Processing: yolo_base=yolo_nas_l, config=1a_angle_t2s, n=256, iteration=14
Processing: yolo_base=yolo_nas_l,

Processing: yolo_base=yolo_nas_l, config=1b_angle_s2t, n=64, iteration=2
Processing: yolo_base=yolo_nas_l, config=1b_angle_s2t, n=64, iteration=3
Processing: yolo_base=yolo_nas_l, config=1b_angle_s2t, n=64, iteration=4
Processing: yolo_base=yolo_nas_l, config=1b_angle_s2t, n=64, iteration=5
Processing: yolo_base=yolo_nas_l, config=1b_angle_s2t, n=64, iteration=6
Processing: yolo_base=yolo_nas_l, config=1b_angle_s2t, n=64, iteration=7
Processing: yolo_base=yolo_nas_l, config=1b_angle_s2t, n=64, iteration=8
Processing: yolo_base=yolo_nas_l, config=1b_angle_s2t, n=64, iteration=9
Processing: yolo_base=yolo_nas_l, config=1b_angle_s2t, n=64, iteration=10
Processing: yolo_base=yolo_nas_l, config=1b_angle_s2t, n=64, iteration=11
Processing: yolo_base=yolo_nas_l, config=1b_angle_s2t, n=64, iteration=12
Processing: yolo_base=yolo_nas_l, config=1b_angle_s2t, n=64, iteration=13
Processing: yolo_base=yolo_nas_l, config=1b_angle_s2t, n=64, iteration=14
Processing: yolo_base=yolo_nas_l, config=1b_an

Processing: yolo_base=yolo_nas_l, config=1b_angle_s2t, n=500, iteration=2
Processing: yolo_base=yolo_nas_l, config=1b_angle_s2t, n=500, iteration=3
Processing: yolo_base=yolo_nas_l, config=1b_angle_s2t, n=500, iteration=4
Processing: yolo_base=yolo_nas_l, config=1b_angle_s2t, n=500, iteration=5
Processing: yolo_base=yolo_nas_l, config=1b_angle_s2t, n=500, iteration=6
Processing: yolo_base=yolo_nas_l, config=1b_angle_s2t, n=500, iteration=7
Processing: yolo_base=yolo_nas_l, config=1b_angle_s2t, n=500, iteration=8
Processing: yolo_base=yolo_nas_l, config=1b_angle_s2t, n=500, iteration=9
Processing: yolo_base=yolo_nas_l, config=1b_angle_s2t, n=500, iteration=10
Processing: yolo_base=yolo_nas_l, config=1b_angle_s2t, n=500, iteration=11
Processing: yolo_base=yolo_nas_l, config=1b_angle_s2t, n=500, iteration=12
Processing: yolo_base=yolo_nas_l, config=1b_angle_s2t, n=500, iteration=13
Processing: yolo_base=yolo_nas_l, config=1b_angle_s2t, n=500, iteration=14
Processing: yolo_base=yolo_nas_l,

Processing: yolo_base=yolo_nas_l, config=2_light, n=128, iteration=2
Processing: yolo_base=yolo_nas_l, config=2_light, n=128, iteration=3
Processing: yolo_base=yolo_nas_l, config=2_light, n=128, iteration=4
Processing: yolo_base=yolo_nas_l, config=2_light, n=128, iteration=5
Processing: yolo_base=yolo_nas_l, config=2_light, n=128, iteration=6
Processing: yolo_base=yolo_nas_l, config=2_light, n=128, iteration=7
Processing: yolo_base=yolo_nas_l, config=2_light, n=128, iteration=8
Processing: yolo_base=yolo_nas_l, config=2_light, n=128, iteration=9
Processing: yolo_base=yolo_nas_l, config=2_light, n=128, iteration=10
Processing: yolo_base=yolo_nas_l, config=2_light, n=128, iteration=11
Processing: yolo_base=yolo_nas_l, config=2_light, n=128, iteration=12
Processing: yolo_base=yolo_nas_l, config=2_light, n=128, iteration=13
Processing: yolo_base=yolo_nas_l, config=2_light, n=128, iteration=14
Processing: yolo_base=yolo_nas_l, config=2_light, n=128, iteration=15
Processing: yolo_base=yolo_n

...

Processing: yolo_base=yolo_nas_s, config=1b_angle_s2t, n=500, iteration=2
Processing: yolo_base=yolo_nas_s, config=1b_angle_s2t, n=500, iteration=3
Processing: yolo_base=yolo_nas_s, config=1b_angle_s2t, n=500, iteration=4
Processing: yolo_base=yolo_nas_s, config=1b_angle_s2t, n=500, iteration=5
Processing: yolo_base=yolo_nas_s, config=1b_angle_s2t, n=500, iteration=6
Processing: yolo_base=yolo_nas_s, config=1b_angle_s2t, n=500, iteration=7
Processing: yolo_base=yolo_nas_s, config=1b_angle_s2t, n=500, iteration=8
Processing: yolo_base=yolo_nas_s, config=1b_angle_s2t, n=500, iteration=9
Processing: yolo_base=yolo_nas_s, config=1b_angle_s2t, n=500, iteration=10
Processing: yolo_base=yolo_nas_s, config=1b_angle_s2t, n=500, iteration=11
Processing: yolo_base=yolo_nas_s, config=1b_angle_s2t, n=500, iteration=12
Processing: yolo_base=yolo_nas_s, config=1b_angle_s2t, n=500, iteration=13
Processing: yolo_base=yolo_nas_s, config=1b_angle_s2t, n=500, iteration=14
Processing: yolo_base=yolo_nas_s,

Processing: yolo_base=yolo_nas_s, config=2_light, n=128, iteration=2
Processing: yolo_base=yolo_nas_s, config=2_light, n=128, iteration=3
Processing: yolo_base=yolo_nas_s, config=2_light, n=128, iteration=4
Processing: yolo_base=yolo_nas_s, config=2_light, n=128, iteration=5
Processing: yolo_base=yolo_nas_s, config=2_light, n=128, iteration=6
Processing: yolo_base=yolo_nas_s, config=2_light, n=128, iteration=7
Processing: yolo_base=yolo_nas_s, config=2_light, n=128, iteration=8
Processing: yolo_base=yolo_nas_s, config=2_light, n=128, iteration=9
Processing: yolo_base=yolo_nas_s, config=2_light, n=128, iteration=10
Processing: yolo_base=yolo_nas_s, config=2_light, n=128, iteration=11
Processing: yolo_base=yolo_nas_s, config=2_light, n=128, iteration=12
Processing: yolo_base=yolo_nas_s, config=2_light, n=128, iteration=13
Processing: yolo_base=yolo_nas_s, config=2_light, n=128, iteration=14
Processing: yolo_base=yolo_nas_s, config=2_light, n=128, iteration=15
Processing: yolo_base=yolo_n

Processing: yolo_base=yolo_nas_s, config=3_breed, n=32, iteration=2
Processing: yolo_base=yolo_nas_s, config=3_breed, n=32, iteration=3
Processing: yolo_base=yolo_nas_s, config=3_breed, n=32, iteration=4
Processing: yolo_base=yolo_nas_s, config=3_breed, n=32, iteration=5
Processing: yolo_base=yolo_nas_s, config=3_breed, n=32, iteration=6
Processing: yolo_base=yolo_nas_s, config=3_breed, n=32, iteration=7
Processing: yolo_base=yolo_nas_s, config=3_breed, n=32, iteration=8
Processing: yolo_base=yolo_nas_s, config=3_breed, n=32, iteration=9
Processing: yolo_base=yolo_nas_s, config=3_breed, n=32, iteration=10
Processing: yolo_base=yolo_nas_s, config=3_breed, n=32, iteration=11
Processing: yolo_base=yolo_nas_s, config=3_breed, n=32, iteration=12
Processing: yolo_base=yolo_nas_s, config=3_breed, n=32, iteration=13
Processing: yolo_base=yolo_nas_s, config=3_breed, n=32, iteration=14
Processing: yolo_base=yolo_nas_s, config=3_breed, n=32, iteration=15
Processing: yolo_base=yolo_nas_s, config=3

Processing: yolo_base=yolo_nas_s, config=4_all, n=64, iteration=2
Processing: yolo_base=yolo_nas_s, config=4_all, n=64, iteration=3
Processing: yolo_base=yolo_nas_s, config=4_all, n=64, iteration=4
Processing: yolo_base=yolo_nas_s, config=4_all, n=64, iteration=5
Processing: yolo_base=yolo_nas_s, config=4_all, n=64, iteration=6
Processing: yolo_base=yolo_nas_s, config=4_all, n=64, iteration=7
Processing: yolo_base=yolo_nas_s, config=4_all, n=64, iteration=8
Processing: yolo_base=yolo_nas_s, config=4_all, n=64, iteration=9
Processing: yolo_base=yolo_nas_s, config=4_all, n=64, iteration=10
Processing: yolo_base=yolo_nas_s, config=4_all, n=64, iteration=11
Processing: yolo_base=yolo_nas_s, config=4_all, n=64, iteration=12
Processing: yolo_base=yolo_nas_s, config=4_all, n=64, iteration=13
Processing: yolo_base=yolo_nas_s, config=4_all, n=64, iteration=14
Processing: yolo_base=yolo_nas_s, config=4_all, n=64, iteration=15
Processing: yolo_base=yolo_nas_s, config=4_all, n=64, iteration=16
Pro

Processing: yolo_base=yolo_nas_s, config=4_all, n=500, iteration=2
Processing: yolo_base=yolo_nas_s, config=4_all, n=500, iteration=3
Processing: yolo_base=yolo_nas_s, config=4_all, n=500, iteration=4
Processing: yolo_base=yolo_nas_s, config=4_all, n=500, iteration=5
Processing: yolo_base=yolo_nas_s, config=4_all, n=500, iteration=6
Processing: yolo_base=yolo_nas_s, config=4_all, n=500, iteration=7
Processing: yolo_base=yolo_nas_s, config=4_all, n=500, iteration=8
Processing: yolo_base=yolo_nas_s, config=4_all, n=500, iteration=9
Processing: yolo_base=yolo_nas_s, config=4_all, n=500, iteration=10
Processing: yolo_base=yolo_nas_s, config=4_all, n=500, iteration=11
Processing: yolo_base=yolo_nas_s, config=4_all, n=500, iteration=12
Processing: yolo_base=yolo_nas_s, config=4_all, n=500, iteration=13
Processing: yolo_base=yolo_nas_s, config=4_all, n=500, iteration=14
Processing: yolo_base=yolo_nas_s, config=4_all, n=500, iteration=15
Processing: yolo_base=yolo_nas_s, config=4_all, n=500, i